### 문제 5-1 : 카페 메뉴 도구(Tool) 호출 체인 구현

In [ ]:
# pip install langchain langchain-openai langchain-community langchain-ollama faiss-cpu wikipedia tavily
# %pip install -U langchain-ollama

import os
from dotenv import load_dotenv

# .env 파일 로드
load_dotenv()

# 모델 일관성을 위한 상수 정의
MODEL_NAME = "qwen3:1.7b"
BASE_URL = "http://localhost:11434"
EMBEDDING_MODEL_NAME = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"

In [ ]:
# cafe_menu.txt 파일 생성
menu_data = """
아메리카노: 가격 ₩4,500, 재료 에스프레소, 뜨거운 물, 특징 원두 본연의 맛과 향을 즐길 수 있는 기본적인 커피.
카페 라떼: 가격 ₩5,000, 재료 에스프레소, 스팀 밀크, 특징 부드러운 우유 거품과 에스프레소의 조화가 일품인 커피.
카푸치노: 가격 ₩5,000, 재료 에스프레소, 스팀 밀크, 우유 거품, 특징 풍성한 우유 거품이 매력적인 커피.
바닐라 라떼: 가격 ₩5,500, 재료 에스프레소, 스팀 밀크, 바닐라 시럽, 특징 달콤한 바닐라 향이 더해진 부드러운 라떼.
초코 라떼: 가격 ₩5,500, 재료 초콜릿 소스, 스팀 밀크, 특징 진한 초콜릿의 달콤함을 느낄 수 있는 음료.
녹차 라떼: 가격 ₩5,500, 재료 녹차 파우더, 스팀 밀크, 특징 쌉쌀한 녹차의 맛과 부드러운 우유의 조화.
딸기 스무디: 가격 ₩6,000, 재료 딸기, 요거트, 우유, 얼음, 특징 신선한 딸기의 상큼함과 요거트의 부드러움.
망고 스무디: 가격 ₩6,000, 재료 망고, 요거트, 우유, 얼음, 특징 열대과일 망고의 달콤함을 시원하게 즐기는 음료.
레몬 에이드: 가격 ₩5,000, 재료 레몬, 탄산수, 설탕, 특징 상큼한 레몬과 톡 쏘는 탄산의 조합이 시원한 음료.
자몽 에이드: 가격 ₩5,000, 재료 자몽, 탄산수, 설탕, 특징 쌉쌀하면서도 달콤한 자몽의 맛이 매력적인 음료.
"""

# './data/' 디렉토리 생성 (없으면)
if not os.path.exists('./data'):
    os.makedirs('./data')

with open('./data/cafe_menu.txt', 'w', encoding='utf-8') as f:
    f.write(menu_data.strip())

print("cafe_menu.txt 파일이 './data/' 폴더에 생성되었습니다.")

# 텍스트 데이터를 Document 객체로 분할 및 벡터 DB 구축
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
# Deprecation Warning 해결: langchain_community.embeddings -> langchain_huggingface
from langchain_huggingface.embeddings import HuggingFaceEmbeddings # HuggingFace 모델 사용
from langchain.text_splitter import CharacterTextSplitter
import os

# 데이터 로드
loader = TextLoader("./data/cafe_menu.txt", encoding='utf-8')
documents = loader.load()

# 텍스트 분할
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

# 임베딩 모델 로드 (HuggingFace의 sentence-transformers 사용)
embeddings = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL_NAME)

# FAISS 벡터 인덱스 생성 및 저장
db_path = "./db/cafe_db"
if not os.path.exists(db_path):
    os.makedirs(db_path)

db = FAISS.from_documents(docs, embeddings)
db.save_local(db_path)

print(f"FAISS 벡터 DB가 '{db_path}' 경로에 생성 및 저장되었습니다.")

In [ ]:
from langchain_tavily import TavilySearch
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain.agents import tool
from langchain_ollama import ChatOllama # ChatOllama 사용
from langchain_core.messages import HumanMessage
from langchain_core.runnables import RunnableLambda

# Tavily API 키 확인
tavily_api_key = os.getenv("TAVILY_API_KEY")
if not tavily_api_key:
    raise ValueError("TAVILY_API_KEY 환경 변수가 설정되지 않았습니다. .env 파일에 TAVILY_API_KEY를 설정해주세요.")

# a) tavily_search_func 정의 (TavilySearchResults -> TavilySearch)
tavily_search = TavilySearch(max_results=5) # TavilySearch로 변경
tavily_search_func = tavily_search.run

# b) wiki_summary 정의
# Wikipedia API 래퍼 초기화
wikipedia_api_wrapper = WikipediaAPIWrapper(top_k_results=3, doc_content_chars_max=500)
wiki_summary = WikipediaQueryRun(api_wrapper=wikipedia_api_wrapper)
wiki_summary_func = wiki_summary.run


# c) db_search_cafe_func 정의
# 벡터 DB 로드
embeddings = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL_NAME) # 상수 활용
db_path = "./db/cafe_db"
local_db = FAISS.load_local(db_path, embeddings, allow_dangerous_deserialization=True)

@tool
def db_search_cafe_func(query: str) -> str:
    """
    로컬 카페 메뉴 DB에서 정보를 검색합니다.
    예: '아메리카노 가격', '카페 라떼 재료', '딸기 스무디 설명'
    """
    docs = local_db.similarity_search(query, k=1) # 가장 유사한 1개 문서 반환
    if docs:
        return docs[0].page_content
    return "해당 메뉴 정보를 찾을 수 없습니다."

print("세 가지 도구(tavily_search_func, wiki_summary, db_search_cafe_func)가 정의되었습니다.")

# LLM 초기화 (ChatOllama 사용)
# MODEL_NAME 및 BASE_URL 상수 활용
llm = ChatOllama(
    model=MODEL_NAME, # 첫 번째 셀에서 정의한 새로운 MODEL_NAME 사용
    base_url=BASE_URL
)

# LLM에 도구 바인딩
tools = [tavily_search, wiki_summary, db_search_cafe_func]
llm_with_tools = llm.bind_tools(tools)

print("LLM에 도구가 성공적으로 바인딩되었습니다.")

In [ ]:
from langchain_core.runnables import chain as runnable_chain
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage
import json # JSON 문자열 파싱을 위해 추가

# 도구 실행을 위한 헬퍼 함수
def _run_tool_calls(tool_calls):
    if not tool_calls:
        return [] # 빈 리스트 반환
    tool_outputs = []
    for tool_call in tool_calls:
        # tool_call이 딕셔너리 형태일 경우를 처리
        tool_name = None
        tool_args = {}
        tool_call_id = None # tool_call_id도 추출
        
        # Ollama의 tool_calls는 일반적으로 'function'이라는 키 아래에 'name'과 'arguments'를 가집니다.
        # 'arguments'는 JSON 문자열일 수 있으므로 파싱이 필요합니다.
        if isinstance(tool_call, dict) and "function" in tool_call:
            tool_name = tool_call["function"]["name"]
            tool_call_id = tool_call.get("id") # tool_call_id 추출
            try:
                # 'arguments'가 JSON 문자열일 수 있으므로 파싱 시도
                tool_args_str = tool_call["function"].get("arguments", "{}")
                tool_args = json.loads(tool_args_str)
            except json.JSONDecodeError:
                # JSON 파싱 실패 시, arguments를 문자열 그대로 사용하거나 빈 딕셔너리로 처리
                tool_args = {"input": tool_args_str} # 적절히 처리 필요
                print(f"Warning: Failed to parse tool arguments as JSON for tool {tool_name}. Args: {tool_args_str}")
        elif hasattr(tool_call, 'tool') and hasattr(tool_call, 'args'): # LangChain의 ToolCall 객체인 경우
            tool_name = tool_call.tool
            tool_args = tool_call.args
            tool_call_id = tool_call.id
        else:
            print(f"Warning: Unexpected tool_call format: {type(tool_call)} - {tool_call}")
            continue # 처리할 수 없는 형식은 건너뜝니다.

        output = None
        if tool_name == "TavilySearch":
            # TavilySearch는 일반적으로 단일 문자열 인수를 받으므로, tool_args에서 첫 번째 값을 사용합니다.
            # 예: {'query': '검색어'}
            query_arg = tool_args.get("query")
            if query_arg:
                output = tavily_search_func(query_arg)
            else:
                print(f"Warning: TavilySearch called without 'query' argument: {tool_args}")
                output = "TavilySearch: Missing query argument."
        elif tool_name == "WikipediaQueryRun":
            query_arg = tool_args.get("query") or tool_args.get("topic") # Wikipedia는 'query'나 'topic'을 사용할 수 있습니다.
            if query_arg:
                output = wiki_summary_func(query_arg)
            else:
                print(f"Warning: WikipediaQueryRun called without query/topic argument: {tool_args}")
                output = "WikipediaQueryRun: Missing query/topic argument."
        elif tool_name == "db_search_cafe_func":
            query_arg = tool_args.get("query") # db_search_cafe_func는 'query' 인수를 받습니다.
            if query_arg:
                output = db_search_cafe_func(query_arg)
            else:
                print(f"Warning: db_search_cafe_func called without 'query' argument: {tool_args}")
                output = "db_search_cafe_func: Missing query argument."
        else:
            print(f"Warning: Unknown tool name called: {tool_name}")
            output = f"Unknown tool: {tool_name}"
            
        tool_outputs.append({"tool_call_id": tool_call_id, "output": output})
    return tool_outputs

@runnable_chain
def tool_calling_chain(query: str):
    # 1. LLM이 도구를 선택하도록 질의
    response: AIMessage = llm_with_tools.invoke(query)

    # 2. tool_calls 속성을 통해 도구 호출 결과 확인
    if response.tool_calls:
        # 3. 도구 실행
        tool_outputs = _run_tool_calls(response.tool_calls)

        # 4. 도구 실행 결과를 LLM에 다시 전달하여 최종 답변 생성
        messages = [
            HumanMessage(content=query),
            response, # 이전 LLM 응답 (tool_calls 포함)
        ]
        # 각 도구 호출 결과에 대해 ToolMessage를 추가
        for tool_output in tool_outputs:
            if tool_output['tool_call_id']: # tool_call_id가 있을 때만 ToolMessage 추가
                messages.append(ToolMessage(
                    content=str(tool_output['output']),
                    tool_call_id=tool_output['tool_call_id']
                ))
            else:
                # tool_call_id가 없으면 일반 HumanMessage로 추가하여 LLM이 참고하도록 함
                messages.append(HumanMessage(content=f"Tool output: {tool_output['output']}"))

        # 이제 업데이트된 메시지 리스트를 LLM에 전달하여 최종 답변을 받습니다.
        final_answer = llm.invoke(messages)
        return final_answer.content
    else:
        # LLM이 도구를 호출하지 않고 직접 답변할 경우
        return response.content

print("간단한 도구 호출 체인이 구현되었습니다.")

In [ ]:
# 테스트 질문
question = "아메리카노의 가격과 특징은 무엇인가요?"

print(f"질문: {question}")

# 체인 실행
# tool_calling_chain의 invoke는 RunnableLambda를 통해 직접 호출되므로,
# _run_tool_calls 함수는 내부적으로 호출됩니다.
# 위 구현된 tool_calling_chain은 첫번째 invoke가 tool_calls를 반환하면,
# 두번째 invoke (final_answer_prompt)를 통해 최종 답변을 생성합니다.
# 직접적으로 최종 답변을 반환하도록 수정했습니다.

final_answer = tool_calling_chain.invoke(question)

print("\n--- 최종 답변 ---")
print(final_answer)

# 예상 처리 과정 (재확인)
# 1. LLM이 질문 분석 -> 메뉴 정보 필요 판단
# 2. db_search_cafe_func 도구 호출 (내부적으로)
# 3. 벡터 DB에서 "아메리카노" 관련 정보 검색
# 4. 가격(₩4,500), 재료(에스프레소, 뜨거운 물), 특징(원두 본연의 맛) 정보 반환
# 5. 정보를 자연어로 정리하여 사용자에게 답변

### 문제 5-2 : Few-shot 프롬프팅을 활용한 카페 AI 어시스턴트

In [29]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage
# ToolCall 객체를 명시적으로 임포트합니다.
from langchain_core.messages import ToolCall
import json # arguments를 파싱할 때 필요하므로 임포트합니다.

# 시스템 메시지 (2번 요구사항 포함)
SYSTEM_MESSAGE = """
당신은 카페 메뉴 정보와 일반적인 음식/음료 지식을 제공하는 AI입니다.

도구 사용 가이드라인:
- db_search_cafe_func: 카페 메뉴 정보 (가격, 재료, 설명) 검색에 사용됩니다. 예를 들어, 특정 메뉴의 가격이나 재료, 설명을 알고 싶을 때 사용하세요.
- wiki_summary: 일반적인 지식 (역사, 제조법, 문화적 배경 등) 검색에 사용됩니다. 예를 들어, 커피의 역사, 특정 음료의 기원, 재료의 일반적인 정보 등을 찾을 때 사용하세요.
- TavilySearch: 최신 정보 (트렌드, 뉴스, 실시간 정보) 검색에 사용됩니다. 예를 들어, 최신 카페 트렌드, 새로운 음료 소식, 특정 지역의 카페 위치 등을 찾을 때 사용하세요.

사용 원칙:
1. 카페 메뉴 관련 질문은 반드시 메뉴 DB(db_search_cafe_func)를 먼저 검색합니다.
2. 역사, 문화, 일반 지식 관련 질문은 위키피디아(wiki_summary)를 활용합니다.
3. 최신 트렌드, 뉴스, 실시간 정보 관련 질문은 웹 검색(TavilySearch)을 활용합니다.
4. 복합 질문의 경우, 여러 도구를 순차적으로 사용하여 필요한 모든 정보를 검색하고 종합합니다.
5. 답변 시 정보의 출처(메뉴 DB, 위키피디아, 웹 검색 등)를 명확히 구분하여 알려주세요.
6. 도구 호출이 필요 없는 간단한 질문(예: 인사말)에는 직접 답변합니다.
"""

# Few-shot 예제 대화
# 각 예제는 HumanMessage, AIMessage, ToolMessage의 조합으로 구성됩니다.
FEW_SHOT_EXAMPLES = [
    # 예제 1: 메뉴 정보 + 일반 지식
    HumanMessage(content="아메리카노 정보와 커피 역사를 알려주세요."),
    AIMessage(
        content="메뉴 DB에서 아메리카노 정보를, 위키피디아에서 커피 역사를 검색하겠습니다.",
        tool_calls=[
            # ToolCall 객체로 변경
            ToolCall(
                name="db_search_cafe_func",
                args=json.loads('{"query": "아메리카노"}'), # JSON 문자열을 딕셔너리로 파싱
                id="call_db_1"
            ),
            ToolCall(
                name="wiki_summary",
                args=json.loads('{"query": "커피의 역사"}'), # JSON 문자열을 딕셔너리로 파싱
                id="call_wiki_1"
            )
        ]
    ),
    ToolMessage(content="아메리카노: 가격 ₩4,500, 재료 에스프레소, 뜨거운 물, 특징 원두 본연의 맛과 향을 즐길 수 있는 기본적인 커피.",
                tool_call_id="call_db_1"),
    ToolMessage(content="커피의 역사는 에티오피아에서 시작되어 중동을 거쳐 전 세계로 퍼졌습니다. 17세기에는 유럽에 전파되며 중요한 음료가 되었습니다. (위키피디아 요약)",
                tool_call_id="call_wiki_1"),
    AIMessage(content="아메리카노는 ₩4,500이며, 에스프레소와 뜨거운 물로 만들어져 원두 본연의 맛을 즐길 수 있습니다. 커피의 역사는 에티오피아에서 시작하여 전 세계로 퍼졌으며, 17세기에는 유럽에 전파되었습니다."),

    # 예제 2: 최신 정보 (Tavily Search)
    HumanMessage(content="요즘 유행하는 커피 트렌드가 궁금해요."),
    AIMessage(
        content="최신 커피 트렌드를 웹 검색으로 알아보겠습니다.",
        tool_calls=[
            ToolCall(
                name="TavilySearch",
                args=json.loads('{"query": "최신 커피 트렌드"}'), # JSON 문자열을 딕셔너리로 파싱
                id="call_tavily_1"
            )
        ]
    ),
    ToolMessage(content="2024년에는 스페셜티 커피의 인기가 지속되며, 콜드브루와 대체 우유 옵션이 더욱 다양해지는 추세입니다. 친환경적인 소비와 홈카페 문화도 주목받고 있습니다. (웹 검색 결과)",
                tool_call_id="call_tavily_1"),
    AIMessage(content="최신 커피 트렌드에 따르면, 스페셜티 커피의 인기가 계속되고 콜드브루와 대체 우유 옵션이 다양해지고 있습니다. 친환경 소비와 홈카페 문화도 주목할 만합니다."),

    # 예제 3: 단일 메뉴 정보 (db_search_cafe_func)
    HumanMessage(content="바닐라 라떼 가격은 얼마인가요?"),
    AIMessage(
        content="바닐라 라떼의 가격을 메뉴 DB에서 확인하겠습니다.",
        tool_calls=[
            ToolCall(
                name="db_search_cafe_func",
                args=json.loads('{"query": "바닐라 라떼 가격"}'), # JSON 문자열을 딕셔너리로 파싱
                id="call_db_2"
            )
        ]
    ),
    ToolMessage(content="바닐라 라떼: 가격 ₩5,500, 재료 에스프레소, 스팀 밀크, 바닐라 시럽, 특징 달콤한 바닐라 향이 더해진 부드러운 라떼.",
                tool_call_id="call_db_2"),
    AIMessage(content="바닐라 라떼의 가격은 ₩5,500입니다. 에스프레소, 스팀 밀크, 바닐라 시럽으로 만들어집니다."),
]

# 프롬프트 템플릿 정의
PROMPT_TEMPLATE = ChatPromptTemplate.from_messages([
    ("system", SYSTEM_MESSAGE),
    MessagesPlaceholder("examples"),
    ("human", "{question}"),
    MessagesPlaceholder("agent_scratchpad")
])

print("Few-shot 예제를 포함한 프롬프트 템플릿이 작성되었습니다.")

Few-shot 예제를 포함한 프롬프트 템플릿이 작성되었습니다.


In [40]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, ToolMessage
import json

# (이전에 정의된 MODEL_NAME, BASE_URL, EMBEDDING_MODEL_NAME 상수,
# 그리고 도구 함수와 llm_with_tools가 정의되어 있어야 합니다.)

# 도구 실행을 위한 헬퍼 함수 (이전과 동일)
def _run_tool_calls(tool_calls):
    if not tool_calls:
        return []
    tool_outputs = []
    for tool_call in tool_calls:
        tool_name = None
        tool_args = {}
        tool_call_id = None
        
        if isinstance(tool_call, dict) and "function" in tool_call:
            tool_name = tool_call["function"]["name"]
            tool_call_id = tool_call.get("id")
            try:
                tool_args_str = tool_call["function"].get("arguments", "{}")
                tool_args = json.loads(tool_args_str)
            except json.JSONDecodeError:
                tool_args = {"input": tool_args_str}
                print(f"경고: 도구 {tool_name}의 인자를 JSON으로 파싱하는 데 실패했습니다. Args: {tool_args_str}")
        elif hasattr(tool_call, 'name') and hasattr(tool_call, 'args'):
            tool_name = tool_call.name
            tool_args = tool_call.args
            tool_call_id = tool_call.id
        else:
            print(f"경고: 예상치 못한 tool_call 형식: {type(tool_call)} - {tool_call}")
            continue

        output = None
        if tool_name == "TavilySearch":
            query_arg = tool_args.get("query")
            if query_arg:
                output = tavily_search_func(query_arg)
            else:
                print(f"경고: 'query' 인자 없이 TavilySearch가 호출되었습니다: {tool_args}")
                output = "TavilySearch: 쿼리 인자 누락."
        elif tool_name == "WikipediaQueryRun":
            query_arg = tool_args.get("query") or tool_args.get("topic")
            if query_arg:
                output = wiki_summary_func(query_arg)
            else:
                print(f"경고: 'query' 또는 'topic' 인자 없이 WikipediaQueryRun이 호출되었습니다: {tool_args}")
                output = "WikipediaQueryRun: 쿼리/토픽 인자 누락."
        elif tool_name == "db_search_cafe_func":
            query_arg = tool_args.get("query")
            if query_arg:
                output = db_search_cafe_func(query_arg)
            else:
                print(f"경고: 'query' 인자 없이 db_search_cafe_func가 호출되었습니다: {tool_args}")
                output = "db_search_cafe_func: 쿼리 인자 누락."
        else:
            print(f"경고: 알 수 없는 도구 이름이 호출되었습니다: {tool_name}")
            output = f"알 수 없는 도구: {tool_name}"
            
        tool_outputs.append({"tool_call_id": tool_call_id, "output": output, "tool_name": tool_name})
    return tool_outputs

# 첫 번째 LLM 호출을 위한 체인 정의 (도구 선택)
# 이 체인은 {"question": ..., "chat_history": ...} 를 입력으로 받아
# PROMPT_TEMPLATE를 구성하고, LLM을 호출하여 AIMessage를 반환합니다.
first_llm_call_chain = (
    RunnablePassthrough.assign(
        agent_scratchpad=lambda x: [
            m for m in x["chat_history"] if isinstance(m, (AIMessage, ToolMessage))
        ],
        question=lambda x: x["question"] 
    )
    | PROMPT_TEMPLATE.partial(examples=FEW_SHOT_EXAMPLES)
    | llm_with_tools # 이 부분의 출력은 AIMessage 객체입니다.
)

# 도구 실행 및 최종 답변 생성 로직
# 이 함수는 {"question": ..., "chat_history": ..., "llm_output": AIMessage} 형태의 입력을 받습니다.
def _execute_tools_and_respond(input_data):
    # input_data는 {"question": OriginalQuestion, "chat_history": CurrentChatHistory, "llm_output": AIMessage_Object} 형태
    response: AIMessage = input_data["llm_output"] # AIMessage 객체를 올바르게 가져옵니다.
    current_chat_history = input_data["chat_history"]
    original_question = input_data["question"]

    if response.tool_calls:
        print(f"\n--- LLM이 호출한 도구: {response.tool_calls} ---")
        tool_outputs = _run_tool_calls(response.tool_calls)
        print(f"--- 도구 실행 결과: {tool_outputs} ---")

        messages_for_final_llm_call = [
            HumanMessage(content=original_question),
            response # LLM의 초기 응답 (도구 호출 포함)
        ]
        
        for tool_output in tool_outputs:
            if tool_output['tool_call_id']:
                messages_for_final_llm_call.append(ToolMessage(
                    content=str(tool_output['output']),
                    tool_call_id=tool_output['tool_call_id']
                ))
            else:
                messages_for_final_llm_call.append(HumanMessage(content=f"도구 결과 ({tool_output.get('tool_name', '알 수 없는 도구')}): {tool_output['output']}"))

        final_answer_response = PROMPT_TEMPLATE.invoke(
            {"question": original_question, "examples": FEW_SHOT_EXAMPLES, "agent_scratchpad": messages_for_final_llm_call}
        )
        return final_answer_response.content
    else:
        print("--- LLM이 직접 답변합니다 ---")
        return response.content

# **수정된 부분**: 최종 파이프라인 구성
# 이 체인은 {"question": ..., "chat_history": ...}를 입력으로 받습니다.
full_chain = (
    # 1. 원본 입력(question, chat_history)을 다음 단계로 전달합니다.
    RunnablePassthrough()
    # 2. 첫 번째 LLM 호출을 실행하고 그 결과를 'llm_output' 키에 할당합니다.
    #    이때, 입력 딕셔너리 전체를 유지하고 'llm_output'만 추가합니다.
    .assign(llm_output=first_llm_call_chain)
    # 3. 할당된 결과를 _execute_tools_and_respond 함수에 전달하여 도구 실행 및 최종 답변 생성
    | RunnableLambda(_execute_tools_and_respond)
)

print("고급 도구 호출 체인(full_chain)이 구현되었습니다.")

# 대화 기록 저장을 위한 Store (메모리 내)
store = {}
def get_session_history(session_id: str) -> ChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

# RunnableWithMessageHistory로 체인 감싸기
final_chain_with_history = RunnableWithMessageHistory(
    full_chain,
    get_session_history,
    input_messages_key="question", # 사용자의 질문이 여기에 매핑됩니다.
    history_messages_key="chat_history" # 대화 기록이 여기에 매핑됩니다.
)

print("체인이 대화 기록 관리 기능을 포함하도록 래핑되었습니다.")

고급 도구 호출 체인(full_chain)이 구현되었습니다.
체인이 대화 기록 관리 기능을 포함하도록 래핑되었습니다.


In [41]:
# 질문 정의
question = "아메리카노 정보와 커피 역사를 알려주세요."
# question = "요즘 유행하는 커피 트렌드가 궁금해요."
# question = "바닐라 라떼 가격은 얼마인가요?"

print(f"질문: {question}")

# 체인 실행
# session_id를 사용하여 대화 기록을 추적합니다.
# **수정된 부분: invoke 호출 시 chat_history 인자 제거**
response_content = final_chain_with_history.invoke(
    {"question": question}, # chat_history는 RunnableWithMessageHistory가 자동으로 처리
    config={"configurable": {"session_id": "test_session_1"}}
)

print("\n--- 최종 답변 ---")
print(response_content)

질문: 아메리카노 정보와 커피 역사를 알려주세요.
--- LLM이 직접 답변합니다 ---

--- 최종 답변 ---
<think>
Okay, the user is asking for information about American coffee and the history of coffee. Let me start by recalling the previous interactions. They first asked about American coffee, and I provided the menu details and the history from Wikipedia. Then they asked about current coffee trends, and I mentioned specialty coffee, cold brew, and eco-friendly trends. Now, they're asking for the same information again, but maybe they want a more detailed or updated answer.

Wait, the user might be checking if the information is still accurate or if there's more to it. Since the history was from Wikipedia, I should confirm that the info is up-to-date. Also, the current trends might have new developments. I need to make sure to include any recent trends or changes in the coffee industry. Let me think about the latest trends in 2024. There's more focus on sustainability, local sourcing, and alternative brewing methods like co